# Исследовательский анализ данных, проектная работа 
# Исследование объявлений о продаже квартир

В нашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Наша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.

### Шаг 1. Откроем файл с данными и изучим общую информацию.

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')

In [ ]:
data.info()

В таблице 22 столбца :

* airports_nearest — расстояние до ближайшего аэропорта в метрах (м)
* balcony — число балконов
* ceiling_height — высота потолков (м)
* cityCenters_nearest — расстояние до центра города (м)
* days_exposition — сколько дней было размещено объявление (от публикации до снятия)
* first_day_exposition — дата публикации
* floor — этаж
* floors_total — всего этажей в доме
* is_apartment — апартаменты (булев тип) - нежилые помещения, не относящиеся к жилому фонду, но имеющие необходимые условия для проживания
* kitchen_area — площадь кухни в квадратных метрах (м²)
* last_price — цена на момент снятия с публикации
* living_area — жилая площадь в квадратных метрах(м²)
* locality_name — название населённого пункта
* open_plan — свободная планировка (булев тип)
* parks_around3000 — число парков в радиусе 3 км
* parks_nearest — расстояние до ближайшего парка (м)
* ponds_around3000 — число водоёмов в радиусе 3 км
* ponds_nearest — расстояние до ближайшего водоёма (м)
* rooms — число комнат
* studio — квартира-студия (булев тип)
* total_area — площадь квартиры в квадратных метрах (м²)
* total_images — число фотографий квартиры в объявлении

In [ ]:
data.head(20)

In [ ]:
data.describe()

### Вывод

При знакомстве с данными обнаружили много пропущенных значений,высота потолков в 1 и 100 метров выглядят нереалистичными.

### Шаг 2. Предобработка данных

Для удобной работы с данными заменим тип данных у цены и общей площади на целочисленный.

In [ ]:
data['last_price'] = data['last_price'].astype('int')
data['total_area'] = data['total_area'].astype('int')
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format = '%Y-%m-%d')

Отсутствие числа балконов говорит нам, что их вовсе нет, заменим пропуски на нули и поменяем тип дпнных на целочисленный.

In [ ]:
data['balcony'] = data['balcony'].fillna(0).astype('int')

Пропуски в апартаментах означают, что это не апартаменты, заменим их на False и поменяем тип дпнных на булевой.

In [ ]:
data['is_apartment'] = data['is_apartment'].fillna(False).astype('bool')

Избавимся от аномальных данных.

In [ ]:
data = data.query('ceiling_height != [1.2, 1.75, 1, 100]')

Заполним пропуски в высоте потолков медианными значениями. Предполагаю, что в пропусках нет выдающихся значений и поэтому не уточняется.

In [ ]:
data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].median())

Удалим строки с пропусками в столбцах с общим количеством этажей и с названием населенного пункта. Потеряем 134 строки, несущественно для выборки. Предполагаем, что пропуски возникли в результате невнимательности при заполнении.

In [ ]:
data.dropna(subset=['floors_total'], inplace = True)
data.dropna(subset=['locality_name'], inplace = True)

Столбец с общим количеством этажей приведем к целочисленному типу

In [ ]:
data['floors_total'] = data['floors_total'].astype(int)

In [ ]:
data['living_area'].corr(data['rooms'])

Наблюдается корреляция между числом комнат и жилой площадью. Создадим сводную таблицу с медианными значениями жилой площади в зависимости от числа комнат.

In [ ]:
rooms_pivot = data.pivot_table(index='rooms', values='living_area', aggfunc=['median'])
rooms_pivot.columns = ['median']
rooms_pivot

Заменим пропуски в столбце с жилой площадью на значения из сводной таблицы. Предполагем, что клиенты не стали уточнять эту информацию.

In [ ]:
def living_area(df, rooms_pivot):
    for a, b in rooms_pivot['median'].items(): 
        df.loc[(df.loc[:, 'rooms']== a) &(df.loc[:, 'living_area'].isnull()), 'living_area' ] = b
living_area(data, rooms_pivot)        
data['living_area'] = data['living_area'].astype(int)

Посчитаем значение для каждого пропущенного значения в площади кухни.

In [ ]:
data['kitchen_area'] = data['kitchen_area'].fillna(data['total_area'] - data['living_area'])

В столбце с названием населенных пунктов, есть двоякое написание "поселка" через "е" и через "ё", исправим.

In [ ]:
data['locality_name'] = data['locality_name'].replace('посёлок', 'поселок', regex=True)
data['locality_name'].unique()

Посмотрим, что у нас получилось.

In [ ]:
data.info()

Для столбцов airports_nearest, cityCenters_nearest, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest, days_exposition нет подходящих значений или корреляций для заполнения, поэтому эти пропуски оставляем. Наличие пропусков в days_exposition скорее всего значит, что объявления всё ещё размещены.

### Шаг 3. Посчитаем и добавим в таблицу, новые данные.

Посчитаем и добавим цену за квадратный метр.

In [ ]:
data['price_per_meter'] = (data['last_price'] / data['total_area']).astype(int)

Добавим столбцы с днем недели, месяцем и годом размещения объявления.

In [ ]:
data['weekday'] = data['first_day_exposition'].dt.weekday
data['month'] = data['first_day_exposition'].dt.month
data['year'] = data['first_day_exposition'].dt.year

Добавим колонки отношения жилой площади к общей, и площади кухни к общей.

In [ ]:
data['living_area_and_total'] = data['living_area'] / data['total_area']
data['kitchen_area_and_total'] = data['kitchen_area'] / data['total_area']

Сгруппируем этажи и добавим колонку, где указано, на каком этаже расположена квартира относительно всего здания.

In [ ]:
def floor_type(df):
    floor = df['floor']
    total_floors = df['floors_total']
    if floor == 1:
        return 'первый'
    elif floor == total_floors:
        return 'последний'
    elif 1 < floor < total_floors:
        return 'другой'

data['floor_type'] = data.apply(floor_type, axis = 1)

### Шаг 4. Проведем исследовательский анализ данных.

Изучим  площадь, цены, число комнат, высоту потолков квартир. Пострим гистограммы для каждого параметра.

In [ ]:
data['total_area'].hist(bins=60, range=(0, 300), figsize=(8, 6))

data['total_area'].describe()

In [ ]:
data.boxplot('total_area')

Совсем маленьких квартир почти нет. Есть выбросы свыше 550, возможно, значения и правдивые, но не отображают положение рынка. Очистим от выбросов и создадим новый датафрейм.

In [ ]:
new_data = data.query('total_area < 550')
new_data.boxplot('total_area')

In [ ]:
(new_data['last_price'] / 1000000).hist(bins=60, range=(0, 30), figsize=(8, 6))
new_data['last_price'].describe()

В основном стоимость квартир колеблется в районе 4 миллионов.

In [ ]:
new_data['rooms'].hist(bins=10, range=(0, 10), figsize=(8, 6))
new_data['rooms'].describe()

Основная часть в обьявлениях это однокомнатные и двукомнатные квартиры.

In [ ]:
new_data['ceiling_height'].hist(bins=6, range=(2, 5), figsize=(8, 6))
new_data['ceiling_height'].describe()

У большинства квартир потолки высотой в районе 2.65 метров, но есть и намного больше.

In [ ]:
import matplotlib.pyplot as plt

new_data.boxplot('days_exposition')
plt.ylim(1,1600)
new_data['days_exposition'].describe()

In [ ]:
new_data.plot(y='days_exposition', kind='hist', bins=400, grid=True, figsize=(16, 9))
new_data.plot(y='days_exposition', kind='hist', bins=200, grid=True, range=(0, 100), figsize=(8, 6))

Половина квартир продается за 3 месяца, четверть продаж за полтора месяца. У нас длинный хвост на диаграмме, поэтому среднее значение в полгода скорее можно считать долгими продажами.

Есть выбросы на 7, 30, 45, 60 и 90 дней, эти данные можно удалить, они не повлияют на анализ.

In [ ]:
new_data = new_data.query('days_exposition not in (7,30,60,45.00000000000001, 90.00000000000001)')
new_data = new_data.query('days_exposition < 730')
new_data['days_exposition'].describe()

In [ ]:
new_data.plot(y='days_exposition', kind='hist', bins=400, grid=True, figsize=(16, 9))
new_data.plot(y='days_exposition', kind='hist', bins=200, grid=True, range=(0, 100), figsize=(8, 6))

In [ ]:
new_data['price_per_meter'].corr(new_data['total_area'])

In [ ]:
def plot_constrctor(index):
    (new_data.pivot_table(index=index, values='price_per_meter',
                             aggfunc=['median']).plot(grid=True))

In [ ]:
new_data.plot(kind = 'scatter', y = 'price_per_meter', x = 'total_area', alpha = 0.04)

Прямой зависимости между общей площадью и ценой за метр не наблюдается.

In [ ]:
new_data['price_per_meter'].corr(new_data['rooms'])

In [ ]:
plot_constrctor(new_data['rooms'])

Прямой корреляции не неблюдется, но по графику видно что, квадратный метр дешевле в трехкомнатных квартирах.



In [ ]:
plot_constrctor(new_data['floor_type'])

Квартиры на первом этаже явно самые дешевые.

In [ ]:
new_data.plot(kind = 'hist', y = 'price_per_meter', x = 'cityCenters_nearest')

В центре квартиры намного дороже.

In [ ]:
new_data['price_per_meter'].corr(new_data['weekday'])

In [ ]:
plot_constrctor(new_data['weekday'])

К воскресенью цены снижаются, а в среду наоборот растут.

In [ ]:
new_data['price_per_meter'].corr(new_data['month'])

In [ ]:
plot_constrctor(new_data['month'])

В июне и декабре самые низкие цены. Январь и август самые выгодные месяцы для продажи квартиры.

In [ ]:
new_data['price_per_meter'].corr(new_data['year'])

In [ ]:
plot_constrctor(new_data['year'])

In [ ]:
new_data['year'].value_counts()

За 2014 год объявлений совсем мало, выборка не валидна.

Сделаем таблицу из 10 самых популярных населенных пунктов. 

In [ ]:
new_data_ten = new_data.pivot_table(index='locality_name', values='price_per_meter', aggfunc=['mean', 'count'])
new_data_ten.columns = ['mean', 'count']
new_data_ten = new_data_ten.sort_values('count', ascending=False).head(10)
new_data_ten

Самый дорогой Санкт-Петербург, а самый дешевый из выбранных Выборг.

Сделаем таблицу по Санкт-Петербургу, расстояние до центра округлим до километров.

In [ ]:
new_data_spb = new_data.query('locality_name == "Санкт-Петербург"').dropna()
new_data_spb['to_center_in_km'] = (new_data_spb['cityCenters_nearest']/ 1000).astype(int)
new_data_spb_pivot = new_data_spb.pivot_table(index='to_center_in_km', values='price_per_meter')   
new_data_spb_pivot.plot(grid=True, figsize=(8, 6), title='Зависимость цены от расстояния до центра')
new_data_spb_pivot

На расстоянии в 3км от центра есть сильная просадка в цене. Дальше следует подъем в цене, и на расстоянии в 8 км. от центра, цены падают еще сильнее, будем считать это центром.

In [ ]:
center_spb = new_data_spb.query('to_center_in_km < 8')
center_spb

In [ ]:
new_data['total_area'].hist(bins=60, range=(0, 300), figsize=(8, 6))
center_spb['total_area'].hist(bins=60, range=(0, 300), figsize=(8, 6))

Распределение площади в центре очень схоже с распределением по остальной выборке.

In [ ]:
(new_data['last_price'] / 1000000).hist(bins=60, range=(0, 30), figsize=(8, 6))
(center_spb['last_price'] / 1000000).hist(bins=60, range=(0, 30), figsize=(8, 6))

В центре смещение к более дорогим квартирам.

In [ ]:
new_data['rooms'].hist(bins=10, range=(0, 10), figsize=(8, 6))
center_spb['rooms'].hist(bins=10, range=(0, 10), figsize=(8, 6))

В центре больше многокомнатных квартир.

In [ ]:
new_data['ceiling_height'].hist(bins=6, range=(2, 5), figsize=(8, 6))
center_spb['ceiling_height'].hist(bins=6, range=(2, 5), figsize=(8, 6))

В центре потолки выше, чем на окраинах.

In [ ]:
def plot_constructor_center(index):
    (center_spb.pivot_table(index=index, values='price_per_meter',
                             aggfunc=['median']).plot(grid=True))

In [ ]:
plot_constructor_center(new_data['rooms'])

В центре однокомнатные квартиры стоят существенно дороже, чем в остальной части Питера.

In [ ]:
plot_constructor_center(new_data['floor_type'])

Картина не отличается от остального города, самые дешевые квартиры на первом этаже.

In [ ]:
center_pivot = center_spb.pivot_table(index='to_center_in_km', values='price_per_meter')   
center_pivot.plot(grid=True, figsize=(8, 6), title='Зависимость цены от расстояния до центра')

Цены снижаются по мере удаления от центара.

In [ ]:
plot_constructor_center(new_data['year'])

In [ ]:
plot_constructor_center(new_data['month'])

В центре картина по месяцам сильно отличается от остального города, май самый выгодный месяц для покупки квартиры в центре.

In [ ]:
plot_constructor_center(new_data['weekday'])

Так же как и в остальной части города в воскресенье самые выгодные цены.

### Шаг 5. Общий вывод.

**<font color="maroon">Факторы влияющие на цену квартир:</font>** 

* Больше половины квартир продается по цене до 5 млн. рублей.
* На цену м² существенное влияние оказывает общая площадь квартиры, в квартирах до 100 м² корреляция составляет 0.24, в квартирах большей площадью влияние увеличивается до 0.39.
* м² в трехкомнатных квартирах на 6.73% цены дешевле чем в остальной выборке.
* Этаж тоже оказывает влияние на стоимость квартиры. Самые дешевые квартиры на первом этаже, затем идут квартиры на последнем этаже, а квартиры между ними оказались самыми дорогими. В среднем квартиры на первом этаже дешевле на 13.95% чем цены в общем на рынке.
* Удаленность квартиры от центра существенно влияет на ее стоимость, и является одним из основных факторов влияющих на конечную цену, корреляция составляет -0.31.
* Также наблюдается сезонность, весной - летом цены дешевле, чем осенью - зимой. И в конце недели цены ниже чем среди недели.

**<font color="maroon">Типичные параметры продающихся квартир:</font>** 

* В основном продаются квартиры площадью 40 - 69 м².
* Типичная цена продажи 3.4 - 6.79 млн. рублей.
* Комнаты от 1 до 3.
* Высота потолков 2.6 - 2.7 м.
* Время продажи составляет от 43 до 229 дней.

**<font color="maroon">Для центра параметры отличаются:</font>** 

Центральная часть города, судя по графику цен, имеет радиус примерно 8 км.

* В центре площадь квартир больше и в основном составляет 55 - 106 м².
* Цена от 6.45 до 14.5 млн. рублей.
* Количество комнат в центре тоже более, здесь однокомнатных практически нет, и продаются в основном двух и трех комнатные.
* Потолки 2.65 - 3.2 м.
* Время продажи 61 - 299 дней.